In [ ]:
### I create some positive and negative filters for later use

### NEGATIVE FILTERS
def food_filter(key):
    others = ['Formula, ready-to-feed', 'Formula, prepared from powder', 'Formula, prepared from concentrate', 'Sugar substitutes', 'Not included in a food category']
    baby_food = ['Baby food: yogurt', 'Baby food: snacks and sweets', 'Baby food: meat and dinners', ]
    desserts_and_snacks = ['Ice cream and frozen dairy desserts', 'Milk shakes and other dairy drinks', 'Cakes and pies', 'Candy not containing chocolate', 'Doughnuts, sweet rolls, pastries', 'Crackers, excludes saltines', 'Cookies and brownies', 'Biscuits, muffins, quick breads', 'Pancakes, waffles, French toast', 'Cereal bars', 'Nutrition bars', 'Saltine crackers', 'Pretzels/snack mix', 'Potato chips', 'Candy containing chocolate', 'Pancakes, waffles, French toast']
    drinks = ['Soft drinks', 'Diet soft drinks', 'Flavored or carbonated water', 'Other diet drinks', 'Beer', 'Liquor and cocktails', 'Wine', 'Nutritional beverages', 'Protein and nutritional powders', 'Sport and energy drinks', 'Diet sport and energy drinks']
    sandwiches = ['Burritos and tacos', 'Other sandwiches (single code)', 'Burgers (single code)', 'Egg/breakfast sandwiches (single code)', 'Frankfurter sandwiches (single code)', 'Frankfurter sandwiches (single code)', 'Vegetables on a sandwich']
    prepared_dishes = ['Rolls and buns', 'Egg rolls, dumplings, sushi', 'Pasta mixed dishes, excludes macaroni and cheese', 'Macaroni and cheese', 'Pizza', 'Meat mixed dishes', 'Stir-fry and soy-based sauce mixtures', 'Bean, pea, legume dishes', 'Seafood mixed dishes', 'Rice mixed dishes', 'Fried rice and lo/chow mein', 'Poultry mixed dishes']
    sauces = ['Dips, gravies, other sauces''Pasta sauces, tomato-based', 'Mustard and other condiments', 'Mayonnaise', 'Jams, syrups, toppings']
    milks = ['Lamb, goat, game', 'Human milk', 'Milk, reduced fat', 'Milk, whole', 'Milk, lowfat', 'Milk, nonfat', 'Flavored milk, whole', 'Yogurt, regular', 'Yogurt, Greek']
    cheese = ['Cheese', 'Cottage/ricotta cheese']
    other_animal_products = ['Eggs and omelets', 'Butter and animal fats']
    meats = ['Ground beef', 'Cold cuts and cured meats', 'Bacon', 'Pork', 'Liver and organ meats', 'Frankfurters', 'Sausages']
    chicken = ['Turkey, duck, other poultry', 'Chicken, whole pieces', 'Chicken patties, nuggets and tenders']
    fish = ['Fish', 'Shellfish']

    milk_substitutes = ['Milk substitutes']
    beans = ['Beans, peas, legumes']
    soy_products = ['Processed soy products']
    nuts = ['Nuts and seeds']
    other_veggie_products = ['Peanut butter and jelly sandwiches (single code)', 'Oatmeal']

    animal_products = milks + cheese + other_animal_products + meats + chicken + fish
    veggie_products = milk_substitutes + beans + soy_products + nuts + other_veggie_products


    filters_map = {
                    "others" : others,
                    "baby_food" : baby_food,
                    "desserts_and_snacks" : desserts_and_snacks,
                    "drinks" : drinks,
                    "sandwiches" : sandwiches,
                    "prepared_dishes" : prepared_dishes,
                    "sauces" : sauces,
                    "milks" : milks,
                    "cheese" : cheese,
                    "other_animal_products" : other_animal_products,
                    "meats" : meats,
                    "chicken" : chicken,
                    "fish" : fish,
                    "milk_substitutes" : milk_substitutes,
                    "beans" : beans,
                    "soy_products" : soy_products,
                    "nuts" : nuts,
                    "other_veggie_products" : other_veggie_products,
                    "animal_products" : animal_products,
                    "veggie_products" : veggie_products
                  }
    
    return filters_map[key]


def multiple_filter(keys):
    final_list = []
    for key in keys:
        final_list = final_list + food_filter(key)

    return final_list

In [ ]:
# Empty list of resources
        resources = []

        # Append the chosen column names of the resources df to the resources empty list
        for resource in resources_df.columns[:-1]:
            # checkbox to select the resources
            resources.append(st.sidebar.checkbox(label = resource))

        # Mean or median
        measure = st.sidebar.radio("Measure of center", options = ["Median", "Mean"]).lower()

        if resources.count(True) > 0:
            #### Data transformation
            # Mask for the dataframe
            column_selection = resources_df.columns[:-1][resources]
            # Stats object
            stats_object = md.stats
            # Table data
            stats = stats_object.calculate(resources_df, column_selection)
            # Plot data
            to_plot = stats_object.to_plot(stats)

            #### Creating table/plots
            stats_ = stats.T.reset_index()
            stats_.columns = ["Resource", "Measure", "Animal-based", "Plant-based"]

            header = list(stats_.columns)
            data = stats_.T

            table = go.Figure(data = go.Table(
                                columnwidth = [25, 25, 25, 25],
                                header = dict(values = header,
                                            fill_color = "#3D5475",
                                            align = "left",
                                            font = dict(size = 20, color = "white")),
                            cells = dict(values = data,
                                        fill_color = "#7FAEF5",
                                        align = "left",
                                        font = dict(size = 16),
                                        height = 30))
                            )

            st.write(to_plot)
            for column in column_selection:
                to_plot_ = to_plot[to_plot["Resource"] == column]
                fig = px.bar(to_plot_[to_plot_["Mean_median"] == measure], x = "Resource", y = "Values",
                                color = "Origin", barmode = "group", title = "Test")
                st.write(fig)
            
            #### Data visualization
            
            st.write(table)

In [ ]:
class comparator:
    def __init__(self, foods, daily_intake):
        self.foods = foods
        self.daily_intake = daily_intake
        self.comparison = self.__comparator()

    ####
    def __comparator(self):
        # Merge first foods series with daily intake series
        comparison = pd.merge(self.daily_intake, self.foods[0], how = "outer", left_index = True, right_index = True)

        # If there's more than one item in foods list...
        if len(self.foods) > 1:
            # then merge the rest of the items with the dataframe we just created
            for food in self.foods[1:]:
                comparison = pd.merge(comparison, food, how = "outer", left_index = True, right_index = True)


        # To conclude, iterate over all food elements
        for food in self.foods:
            # Calculate the % of the daily nutrient intake the food provides with
            comparison[f"Relative - {food.name}"] = (comparison.loc[:, food.name] / comparison.loc[:, "Daily Intake"]) * 100

        return comparison

    ####
    def to_plot(self):
        # We get the columns with the relative nutritional values of the foods
        rel_comparison = comparison.iloc[:, -len(foods):]

        # We'll save the dataframes in the following list
        relatives = []

        # Iterate over the columns in comparison
        for column in rel_comparison.columns:
            # Get the Series coresponding to the food column
            rel = rel_comparison.loc[:, column]
            # Get nutrients out of the index
            rel = rel.reset_index()
            # Add a column with the food name
            rel["Food"] = column[11:]
            # Rename the columns for later use
            rel.columns = ["Nutrient", "Comparison", "Food"]
            # add the dataframe to our list
            relatives.append(rel)

        # Once we have all the dataframes, we'll stack them together vertically and return it
        return pd.concat(relatives)

In [ ]:
def rows_filter(df, filter_, positive = True):
    if positive:
        filtered_df = df[df["Category name"].isin(filter_)]
    else:
        filtered_df = df[~df["Category name"].isin(filter_)]
        
    return filtered_df

def multiple_filter(df, filters_, positive = True):
    dfs = []
    if positive:
        for filter_ in filters_:
            filtered_df = rows_filter(df, filter_)
            dfs.append(filtered_df)

        final_df = pd.concat(dfs)

    else:
        final_df = df[~df["Category name"].isin(filters_)]

    return final_df

In [ ]:
class filter_tool:
    def rows_filter(df, filter_, positive = True):
        if positive:
            filtered_df = df[df["Category name"].isin(filter_)]
        else:
            filtered_df = df[~df["Category name"].isin(filter_)]
            
        return filtered_df

    def multiple_filter(df, filters_, positive = True):
        dfs = []
        if positive:
            for filter_ in filters_:
                filtered_df = rows_filter(df, filter_)
                dfs.append(filtered_df)

            final_df = pd.concat(dfs)

        else:
            final_df = df[~df["Category name"].isin(filters_)]

        return final_df